In [ ]:
#Import required packages
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd
from Helper_functions import load_dict

from AI_functions import resnet18,CellDataset_supervised,data_generator
import torch.nn as nn
import torch.optim as optim


In [ ]:
DATASETS=["BoneMarrow_sample1"]#,"Retina_1_2","Colon","Choroid"]
DATASET="Retina_0_0"
DATASET="Retina_1_2"
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\\" +DATASET
outpath = basepath + "\\Outputs"
image_dim=64 #Dim of the final images
nuclear_channel="Ch7"
cellmask_channel="Ch1_mask"
df=pd.read_csv(outpath+"\\cell_info1.csv")
cell_names=df["Cell_ID"].to_numpy()
Prediction_Channels=['Ch07']

In [ ]:
image_dict=load_dict(outpath,cell_names,image_dim)

In [ ]:
image_dict[6898]['Ch1'].max()

In [ ]:
plt.imshow(image_dict[6898]['Ch1'],cmap="Greys",vmin=-20,vmax=50)
plt.axis('off')
plt.savefig(r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Report\Icons\\"+"debris.png",bbox_inches="tight")
plt.show()
plt.imshow(image_dict[50199]['Ch1'],cmap="Greys",vmin=-20,vmax=50)
plt.axis('off')
plt.savefig(r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Report\Icons\\"+"ss.png",bbox_inches="tight")
plt.show()
plt.imshow(image_dict[6882]['Ch1'],cmap="Greys",vmin=-20,vmax=50)
plt.axis('off')
plt.savefig(r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Report\Icons\\"+"dd.png",bbox_inches="tight")
plt.show()
plt.imshow(image_dict[939]['Ch1'],cmap="Greys",vmin=-20,vmax=50)
plt.axis('off')
plt.savefig(r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Report\Icons\\"+"agg.png",bbox_inches="tight")
plt.show()
plt.imshow(image_dict[9760]['Ch1'],cmap="Greys",vmin=-20,vmax=50)
plt.axis('off')
plt.savefig(r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Report\Icons\\"+"photo.png",bbox_inches="tight")
plt.show()

In [ ]:
plt.imshow(image_dict[15000]['Ch1'])

In [ ]:
BUGS=[]
for image in image_dict:
    for i in image_dict[image]:
      #  print(i)
        try: 
            if i==None:
                BUGS.append(image)
            
        except:
            pass

print(len(BUGS))

In [ ]:
Channels=['Ch1']  #Channel to be fed to the NN
images_with_index = []

for image_i in image_dict:
    #print(image_dict[image_i].keys())
    if len(image_dict[image_i].keys())>=len(Channels):
        image=cv.merge([image_dict[image_i][i] for i in Channels])
        images_with_index.append((int(image_i),image))
    else:
        print(image_i)
    
images=np.array([image[1] for image in images_with_index])
names=np.array([image[0] for image in images_with_index])
assert sum(names!=df['Cell_ID'].to_numpy()) ==0  #Check that the order has been preserved
DNA_pos=df["DNA_pos"].to_numpy()
Touches_Boundary=df["Touches_boundary"].to_numpy()
#labels=df[["Intensity_MC_"+channel for channel in Prediction_Channels]].to_numpy()4
labels=df[["Scaled_"+channel for channel in Prediction_Channels]].to_numpy()

Thresh=50
# plt.hist(df["Gradient RMS_M01_Ch01"],bins=200);
# plt.axvline(x=Thresh,color="red")
# plt.show()
idx_to_keep=np.array(DNA_pos==1,dtype=int)+np.array(Touches_Boundary==0,dtype=int)+np.array(df["Gradient RMS_M01_Ch01"]>Thresh,dtype=int)==3 #keep dnapos, no touch boundarym APC and Other
#Filter
images=images[idx_to_keep]
names=names[idx_to_keep]
labels=labels[idx_to_keep]


mini=int(round(abs(np.array(images).min()),0))
images=images+abs(np.array(images).min())
mean=np.array(images).mean()
maxi=np.array(images).max()
std=np.array(images).std()

In [ ]:
# #Scale and normalize the labels for regression
# from Helper_functions import log_pol_scale,log_pol
# labels=log_pol_scale(labels,slope=1,c=1000)



# plt.hist(labels[:,0],bins=100);
# plt.show()
# #plt.hist(labels[:,1],bins=100);
# #plt.show()
# #plt.hist(labels[:,2],bins=100);
# #print(np.mean(labels,axis=0))

In [ ]:
plt.hist(log_pol(labels[:,0],slope=3),bins=100);
plt.show()

In [ ]:
#del image_dict

In [ ]:
#images,labels,names=images[:10000],labels[:10000],names[:10000]

In [ ]:
returned=data_generator(images,labels,names,mini,train_test_split = 0.8,batch_size = 100,sample=False)
train,test,batch_size,mean_loader,std_loader=returned
[train_data,train_data1,train_labels,train_ID]=train
[test_data,test_data1,test_labels,test_ID]=test

In [ ]:
for i in range(3): 
    plt.imshow(images[i])
    plt.colorbar()
    plt.show()

In [ ]:
# def get_mean_std(loader):
#         #https://stackoverflow.com/questions/48818619/pytorch-how-do-the-means-and-stds-get-calculated-in-the-transfer-learning-tutor
#         mean = 0.
#         std = 0.
#         for images, _,_ in loader:
#             images=images[0]
#             batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
#             images = images.view(batch_samples, images.size(1), -1)
#             mean += images.mean(2).sum(0)
#             std += images.std(2).sum(0)

#         mean /= len(loader.dataset)
#         std /= len(loader.dataset)
#         return mean, std



# def data_generator(images,labels,names,mini,train_test_split = 0.8,batch_size = 100,sample=False):
#     #Split data into training and test, just looking at first images now

#     train_data1=images[:int(train_test_split*len(images))]
#     test_data1=images[int(train_test_split*len(images)):]

#     train_labels=labels[:int(train_test_split*len(images))]
#     test_labels=labels[int(train_test_split*len(images)):]

#     train_ID=names[:int(train_test_split*len(images))]
#     test_ID=names[int(train_test_split*len(images)):]

#     #Transform data into tensors, normalize images
#     transform_basic = transforms.Compose(
#         [transforms.ToPILImage(),
#         transforms.ToTensor()
#     ])

#     train_data_basic = CellDataset_supervised(train_data1,train_labels,train_ID, transform_basic)
#     #Create DataLoaders
#     train_loader_basic = DataLoader(train_data_basic, batch_size=100, shuffle=True)
#     mean_loader,std_loader=get_mean_std(train_loader_basic)


#     #Transform data into tensors, normalize images
#     transform_train = transforms.Compose(
#         [transforms.ToPILImage(),transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomVerticalFlip(p=0.5),
#       #  transforms.RandomRotation(degrees=180,fill=mini),
#         transforms.ToTensor(),
#     # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
#     transforms.Normalize(mean=[mean_loader], std=[std_loader])  # for grayscale images

#     ])

#     #Transform data into tensors, normalize images
#     transform_test = transforms.Compose(
#         [transforms.ToPILImage(),
#         transforms.ToTensor(),
#         #transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
#     transforms.Normalize(mean=[mean_loader], std=[std_loader])  # for grayscale images
#     ])

#     train_data = CellDataset_supervised(train_data1,train_labels,train_ID, transform_train)
#     test_data = CellDataset_supervised(test_data1,test_labels,test_ID, transform_test)



#     return [train_data,train_data1,train_labels,train_ID],[test_data,test_data1,test_labels,test_ID],batch_size,mean_loader,std_loader

# #Barlow

In [ ]:
transform_train = transforms.Compose(
    [transforms.ToPILImage(),transforms.RandomHorizontalFlip(p=0.5),transforms.RandomVerticalFlip(p=0.5),transforms.RandomRotation(degrees=18,fill=mini),
    transforms.ToTensor(),
    transforms.Normalize(mean_loader,std_loader)  
])

transform_validation = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean_loader,std_loader)  
])

In [ ]:
# transform_train  = transforms.Compose([
#         transforms.ToPILImage(),
#         transforms.RandomRotation(degrees=180,fill=mini),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[mean_loader], std=[std_loader]),  # for grayscale images
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomVerticalFlip(p=0.5),
#         transforms.Lambda(lambda x: x if 0.5>np.random.rand() else transforms.functional.invert(x)-1),
#         transforms.Lambda(lambda x: x *(1+0.2*np.random.randn()) ),
#     # transforms.Lambda(lambda x: x + (0.1**0.5)*torch.randn(64, 64) )
#         ]) 



In [ ]:
# plt.hist(train_labels,bins=100);
# x=np.linspace(np.min(train_labels),np.max(train_labels),100)
# plt.plot(x,sampler_f(x))

In [ ]:
train_data = CellDataset_supervised(train_data1,train_labels,train_ID, transform_train)
test_data = CellDataset_supervised(test_data1,test_labels, test_ID,transform_validation)

from torch.utils.data.sampler import WeightedRandomSampler
print(np.max(train_labels))
def sampler_f(x):
    return ((x)+abs(np.min(train_labels)))**3+1
weights=sampler_f(train_labels)
#weights/=np.sum(weights)
torch.tensor(weights)
sampler = WeightedRandomSampler(weights.T[0], len(weights))


#Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size,shuffle=False,sampler=sampler)
validation_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
#torch.save(train_loader,outpath+"\\"+DATASET+"_train.pth")
#torch.save(validation_loader,outpath+"\\"+DATASET+"_validation.pth")

In [ ]:
plt.imshow(next(iter(train_loader))[0][0][0],vmin=-10,vmax=10)
plt.colorbar()
plt.show()
plt.imshow(next(iter(train_loader))[0][0][0],vmin=-10,vmax=10)
plt.colorbar()

In [ ]:
def train_epoch(NN, device, dataloader, loss_fn, optimizer,noise_factor=0):
    NN.train()
    train_loss = []
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch,labels_batch,_ in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        image_noisy = image_batch
        image_batch = image_noisy.to(device)
        labels_batch=labels_batch.to(device)
        output = NN(image_batch)
        # Evaluate loss
        loss = loss_fn(labels_batch,output)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.detach().cpu().numpy())
    return np.mean(train_loss)

### validationing function
def validation_epoch(NN, device, dataloader, loss_fn):
    NN.eval()
    val_loss=[]
    with torch.no_grad(): # No need to track the gradients
        for image_batch,labels_batch,ID_batch in dataloader:
            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            labels_batch = labels_batch.to(device)
            output = NN(image_batch)
            loss = loss_fn(labels_batch,output,)
            val_loss.append(loss.detach().cpu().numpy())

    return np.mean(val_loss)


In [ ]:
df100=pd.DataFrame()
df100['Cell_ID']=np.zeros((len(test_data1)))
for channel in Prediction_Channels:
    df100["AVG_Intensity_MC_"+channel]=np.zeros((len(test_data1)))

def predict(df100):
    predictions=np.zeros((len(test_data1),num_classes))
    names=np.zeros((len(test_data1)))
    labels=np.zeros((len(test_data1),num_classes))
    i=0
    ConvNet_simple.eval()
    with torch.no_grad():
        for X, Y,ID in validation_loader:
            X = X.to(device)
            latents=ConvNet_simple(X).detach().cpu().numpy()
            labels[i:i+len(latents)]=Y
            names[i:i+len(latents)]=ID
            predictions[i:i+len(latents)]=latents
            i+=len(latents)

    df100['Cell_ID']+=names
    for i,channel in enumerate(Prediction_Channels):
        df100["AVG_Intensity_MC_"+channel]+=predictions[:,i]

    return df100

In [ ]:
num_classes=len(Prediction_Channels)


Train_net=torch.load(r'C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Results\\'+"model300loss_tensor(93.1900).pth")
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.fc = nn.Linear(2048, 512)
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fcf = nn.Linear(512, num_classes)
    def forward(self, x):
        e = self.fc(x)
        e1=self.fc1(e)
        e2=self.fc2(e1)
        out=self.fcf(e2)
        return out

Train_net=torch.load(r'C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Results\\'+"model300loss_tensor(93.1900).pth")

net_add=net()
Trained_model = nn.Sequential(Train_net, net_add)  
# for name, param in Trained_model.named_parameters():
#     if param.requires_grad:print(name)

for para in Trained_model[0].parameters():
    para.requires_grad = False

#Trained_model[1].fc.weight.requires_grad=True

#for name, param in Trained_model.named_parameters():
 #   if param.requires_grad:print(name)

In [ ]:
#torch.onnx.export(Train_net,X[0][:,None],'rnn.onnx')

In [ ]:
num_epochs=20
lr=1e-3
num_classes=len(Prediction_Channels)

ConvNet_simple=resnet18(channel_num=len(Channels),num_classes=num_classes)
#ConvNet_simple=Trained_model

device = "cuda" if torch.cuda.is_available() else "cpu"
ConvNet_simple.to(device)
loss_dict={"L1":nn.L1Loss(),"MSE":nn.MSELoss()}

loss_fn = loss_dict["L1"]

#optimizer = optim.Adam(ConvNet_simple.parameters(), lr = lr) 
optimizer = optim.Adam(filter(lambda p: p.requires_grad, ConvNet_simple.parameters()),lr=lr)

diz_loss = {'train_loss':[],'val_loss':[]}

counter=0

for epoch in range(num_epochs):

    train_loss = train_epoch(ConvNet_simple,device,train_loader,loss_fn,optimizer)
    val_loss = validation_epoch(ConvNet_simple,device,validation_loader,loss_fn)

    if epoch>3:
        df100=predict(df100)
        counter+=1

    
    print('\n EPOCH',epoch+1,' \t train loss',train_loss,' \t val loss',val_loss)
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(val_loss)


#_ = loss_over_epochs(diz_loss,num_epochs)

In [ ]:
df100=df100/counter

In [ ]:
plt.imshow(next(iter(train_loader))[0][3][0])
plt.colorbar()

In [ ]:
df50=pd.DataFrame(diz_loss)
df50.to_csv(basepath+"/Results/Resnet"+str(diz_loss['train_loss'][-1])+".csv", index = False, header=True)

In [ ]:

predictions=np.zeros((len(test_data1),num_classes))
names=np.zeros((len(test_data1)))
labels=np.zeros((len(test_data1),num_classes))
i=0
ConvNet_simple.eval()
with torch.no_grad():
    for X, Y,ID in validation_loader:
        X = X.to(device)
        latents=ConvNet_simple(X).detach().cpu().numpy()
        labels[i:i+len(latents)]=Y
        names[i:i+len(latents)]=ID
        predictions[i:i+len(latents)]=latents
        i+=len(latents)


In [ ]:
df["Cell_ID"]=pd.to_numeric(df['Cell_ID'])
df10=pd.DataFrame()
df10['Cell_ID']=names

for i,channel in enumerate(Prediction_Channels):
    df10["PRED_Intensity_MC_"+channel]=predictions[:,i]
    df10["SCALED_Intensity_MC_"+channel]=labels[:,i]


df20=pd.merge(df10,df,on="Cell_ID",how='inner')
#df10.to_csv(basepath+"/Results/Resnet"+str(diz_loss['train_loss'][-1])+".csv", index = False, header=True)


In [ ]:
def plot_scatter(df,channel):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    plt.xlim(np.min(df["SCALED_Intensity_MC_"+channel]),np.max(df["SCALED_Intensity_MC_"+channel]))
    plt.ylim(np.min(df["SCALED_Intensity_MC_"+channel]),np.max(df["SCALED_Intensity_MC_"+channel]))
    xy=np.linspace(-10,10,5000)
    ax.set_aspect('equal', adjustable='box')
    x=df["SCALED_Intensity_MC_"+channel]
    y=df["PRED_Intensity_MC_"+channel]
    plt.scatter(x,y,s=1,alpha=0.5)
    print(np.corrcoef(x,y)[0,1])

    plt.scatter(xy,xy,s=0.5,alpha=0.2,color="red")
    plt.xlabel("Actual fluorescence")
    plt.ylabel("Predicted fluorescence")
    plt.title(channel)
    plt.show()

    # fig = plt.figure(figsize=(10,10))
    # ax = fig.add_subplot(111)
    # plt.xlim(np.min(df["SCALED_Intensity_MC_"+channel]),np.max(df["SCALED_Intensity_MC_"+channel]))
    # plt.ylim(np.min(df["SCALED_Intensity_MC_"+channel]),np.max(df["SCALED_Intensity_MC_"+channel]))
    # xy=np.linspace(-10,10,5000)
    # ax.set_aspect('equal', adjustable='box')
    # x=df["SCALED_Intensity_MC_"+channel]
    # y=df["AVG_Intensity_MC_"+channel]
    # plt.scatter(x,y,s=1,alpha=0.5)
    # print(np.corrcoef(x,y)[0,1])

    # plt.scatter(xy,xy,s=0.5,alpha=0.2,color="red")
    # plt.xlabel("Actual fluorescence")
    # plt.ylabel("Predicted fluorescence")
    # plt.title(channel)
    # plt.show()

In [ ]:
df20

In [ ]:
plot_scatter(df10,'Ch07')

In [ ]:
df20=pd.merge(df10,df,on="Cell_ID",how='inner')

In [ ]:
df20